# Installing required libraries

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00


In [ ]:
pip install torch torchvision pytorch-tabnet xgboost scikit-learn

# Gradio deployment

In [ ]:
import gradio as gr
import joblib
import numpy as np
import cv2
from skimage.feature import hog, local_binary_pattern
from skimage.measure import shannon_entropy
from skimage.filters import sobel, gabor
from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from scipy import stats

# ========= Feature Extractor =========
class ImageFeatureExtractor:
    def __init__(self):
        self.hog_params = {'orientations': 8, 'pixels_per_cell': (16, 16), 'cells_per_block': (2, 2)}

    def extract_features(self, img):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (512, 512))
        _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        features = {}
        features['stroke_density'] = np.sum(binary > 0) / (512*512)
        features['contour_count'] = len(contours)

        hog_feat = hog(img, **self.hog_params)
        features['hog_mean'] = np.mean(hog_feat)
        features['hog_skew'] = stats.skew(hog_feat)

        lbp = local_binary_pattern(img, P=8, R=1)
        features['lbp_entropy'] = shannon_entropy(lbp)

        if contours:
            contours_combined = np.vstack(contours)
            x, y = contours_combined.squeeze().T
            features['writing_span_x'] = np.ptp(x)
            features['writing_span_y'] = np.ptp(y)
            features['aspect_ratio'] = features['writing_span_y'] / (features['writing_span_x'] + 1e-6)
        else:
            features.update({'writing_span_x': 0, 'writing_span_y': 0, 'aspect_ratio': 0})

        edges = sobel(binary)
        features['edge_density'] = np.sum(edges > 0.1) / (512*512)

        gabor_filt_real, _ = gabor(img, frequency=0.6)
        features['gabor_energy'] = np.mean(gabor_filt_real**2)

        features['shannon_entropy'] = shannon_entropy(binary)
        features['spatial_entropy'] = self._calc_spatial_entropy(binary)

        if len(contours) > 1:
            areas = [cv2.contourArea(c) for c in contours]
            features['stroke_size_cv'] = np.std(areas) / np.mean(areas)
        else:
            features['stroke_size_cv'] = 0

        return features

    def _calc_spatial_entropy(self, image):
        grid = cv2.resize(image, (8, 8))
        return shannon_entropy(grid)

# ========= Mapping Function =========
def map_to_80_features(image_features_dict, feature_names_80):
    base_features = list(image_features_dict.values())
    mapped = []

    for i in range(len(feature_names_80)):
        base_feat = base_features[i % len(base_features)]
        noise = np.random.normal(loc=0, scale=0.01)
        mapped_value = base_feat + noise
        mapped.append(mapped_value)

    return np.array(mapped).reshape(1, -1)

# ========= 80 Feature Names =========
feature_names_80 = [
    'air_time2', 'max_y_extension2', 'paper_time2', 'total_time2', 'mean_jerk_in_air3',
    'mean_speed_on_paper3', 'pressure_var3', 'total_time3', 'pressure_mean4', 'pressure_var4',
    'air_time5', 'pressure_mean5', 'air_time6', 'pressure_mean6', 'total_time6',
    'air_time7', 'gmrt_in_air7', 'mean_gmrt7', 'mean_speed_in_air7', 'total_time7',
    'air_time8', 'mean_gmrt8', 'mean_speed_on_paper8', 'paper_time8', 'pressure_mean8',
    'total_time8', 'disp_index9', 'mean_jerk_in_air9', 'mean_jerk_on_paper9', 'paper_time9',
    'pressure_mean9', 'total_time9', 'gmrt_on_paper10', 'paper_time10', 'paper_time11',
    'paper_time12', 'total_time12', 'air_time13', 'total_time13', 'mean_gmrt14',
    'air_time15', 'total_time15', 'air_time16', 'total_time16', 'air_time17',
    'gmrt_in_air17', 'mean_gmrt17', 'mean_jerk_in_air17', 'mean_speed_in_air17', 'paper_time17',
    'total_time17', 'total_time18', 'max_y_extension19', 'num_of_pendown19', 'pressure_mean19',
    'pressure_var19', 'disp_index20', 'max_y_extension20', 'paper_time20', 'total_time20',
    'max_x_extension21', 'pressure_mean21', 'air_time22', 'disp_index22', 'paper_time22',
    'total_time22', 'air_time23', 'disp_index23', 'gmrt_in_air23', 'mean_gmrt23',
    'paper_time23', 'total_time23', 'air_time24', 'mean_jerk_on_paper24', 'total_time24',
    'disp_index25', 'max_y_extension25', 'mean_gmrt25', 'mean_speed_in_air25', 'paper_time25'
]

# ========= Load Model =========
tabnet_model = TabNetClassifier()
tabnet_model.load_model('/content/tabnet_fold5_seed42.zip')

model_xgb: XGBClassifier = joblib.load('/content/xgboost_fold5.pkl')
stacker = joblib.load('/content/stacker_fold5.pkl')
scaler = joblib.load('/content/scaler_fold1.pkl')
extractor = ImageFeatureExtractor()

# ========= Prediction Pipeline =========
def predict(image):
    try:
        base_features = extractor.extract_features(image)
        features_80 = map_to_80_features(base_features, feature_names_80)
        input_scaled = scaler.transform(features_80)
        tabnet_probs = tabnet_model.predict_proba(input_scaled)[0]
        xgb_probs = model_xgb.predict_proba(input_scaled)[0]
        # Keep only the probability of class 1 (Alzheimer)
        combined_features = np.array([tabnet_probs[1], xgb_probs[1]])  # Shape (2,)
        prediction = stacker.predict([combined_features])[0]
        return "🧠 Alzheimer Detected" if prediction == 0 else "✅ No Alzheimer Detected"
    except Exception as e:
        print("Prediction error:", e)
        print("TabNet p(1):", tabnet_probs[1])
        print("XGB p(1):", xgb_probs[1])
        print("Combined for stacker:", combined_features)

        return f"Error during prediction: {str(e)}"



# ========= Gradio Interface =========
gr.Interface(
    fn=predict,
    inputs=gr.Image(type="numpy", label="Upload Drawing/Image"),
    outputs=gr.Label(label="Prediction"),
    title="Alzheimer’s Stage Classifier"
).launch()


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c079c1802904d19c71.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
